# Find a place to install a new japanese restaurant

## 1. Libraries to import

In [11]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

## 2. Get the position of the 20 borough of Paris

In [92]:
arr=pd.read_csv('arrondissements.csv',sep=';')
arr.drop(['C_AR','C_ARINSEE','N_SQ_AR','L_AR','N_SQ_CO','SURFACE','PERIMETRE','Geometry','LONGUEUR'],1,inplace=True)
arr.sort_values('OBJECTID',inplace=True,ignore_index=True)
arr=arr[['OBJECTID','L_AROFF','Geometry X Y']]
lat=[float(arr['Geometry X Y'][i].split(',')[0]) for i in range(0,arr.shape[0])]
long=[float(arr['Geometry X Y'][i].split(',')[1]) for i in range(0,arr.shape[0])]
arr.insert(2,'Latitude',lat)
arr.insert(3,'Longitude',long)
arr.drop(['Geometry X Y'],1,inplace=True)
arr.columns=['Number','Name','Latitude','Longitude']

In [93]:
arr

,Number,Name,Latitude,Longitude
0,1,Louvre,48.862563,2.336443
1,2,Bourse,48.868279,2.342803
2,3,Temple,48.862872,2.360001
3,4,Hôtel-de-Ville,48.854341,2.357630
4,5,Panthéon,48.844443,2.350715
5,6,Luxembourg,48.849130,2.332898
6,7,Palais-Bourbon,48.856174,2.312188
7,8,Élysée,48.872721,2.312554
8,9,Opéra,48.877164,2.337458
9,10,Entrepôt,48.876130,2.360728


## 3. Map of Paris

In [95]:
address='Paris'
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of Paris are 48.8566969, 2.3514616.


In [99]:
map_paris = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, n, a in zip(arr['Latitude'], arr['Longitude'], arr['Number'],arr['Name']):
    label = '{}, {}'.format(n, a)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_paris)  
    
map_paris

## 4. Connection to Foursquare

In [104]:
fs_api=pd.read_csv('foursquare.csv',header=None) #Credentials
CLIENT_ID = fs_api[0][0] # your Foursquare ID
CLIENT_SECRET = fs_api[1][0] # your Foursquare Secret
VERSION = fs_api[2][0] # Foursquare API version

In [105]:
radius = 500
LIMIT = 100

venues = []

for lat, long, n, a in zip(arr['Latitude'], arr['Longitude'], 
                           arr['Number'], arr['Name']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            n, 
            a,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [108]:
venues_df=pd.DataFrame(venues)

In [111]:
venues_df.columns=['Number','Name','Latitude','Longitude',
                   'VenueName','VenueLatitude','VenueLongitude','VenueCategory']

In [112]:
venues_df.head()

,Number,Name,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,1,Louvre,48.862563,2.336443,Musée du Louvre,48.860847,2.336440,Art Museum
1,1,Louvre,48.862563,2.336443,Palais Royal,48.863236,2.337127,Historic Site
2,1,Louvre,48.862563,2.336443,Comédie-Française,48.863088,2.336612,Theater
3,1,Louvre,48.862563,2.336443,Cour Napoléon,48.861172,2.335088,Plaza
4,1,Louvre,48.862563,2.336443,Place du Palais Royal,48.862523,2.336688,Plaza
